<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-packages" data-toc-modified-id="Importing-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing packages</a></span></li><li><span><a href="#Amazon-reviews-scrapper" data-toc-modified-id="Amazon-reviews-scrapper-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Amazon reviews scrapper</a></span></li></ul></div>

In [2]:
# https://landingcube.com/amazon-super-url/#:~:text=A%20Super%20URL%20is%20a,exact%20source%20of%20a%20click.
# https://www.sellerlabs.com/blog/super-urls-exposed/
# https://401stblow.wordpress.com/2010/06/26/amazon-associates-links-made-simple/#:~:text=So%20now%20we%20have%20Amazon's,to%20credit%20for%20the%20sale.

In [3]:
# https://www.amazon.in/Hammer-Wireless-Bluetooth-Playtime-Headphone/dp/B07WY5M3KT/ref=lp_14583473031_1_3_sspa?s=electronics&ie=UTF8&qid=1597608129&sr=1-3-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEyOURIRjFRSUlTNUgyJmVuY3J5cHRlZElkPUEwNzM0OTMyMzJFSlUyQzlYSlBJSCZlbmNyeXB0ZWRBZElkPUEwNjMwMTYyMzhJNzNMOTA5WldIOSZ3aWRnZXROYW1lPXNwX2F0Zl9icm93c2UmYWN0aW9uPWNsaWNrUmVkaXJlY3QmZG9Ob3RMb2dDbGljaz10cnVl
# https://www.amazon.in/Hammer-Wireless-Bluetooth-Playtime-Headphone/product-reviews/B07WY5M3KT/ref=lp_14583473031_1_3_sspa?s=electronics&ie=UTF8&qid=1597608129&sr=1-3-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEyOURIRjFRSUlTNUgyJmVuY3J5cHRlZElkPUEwNzM0OTMyMzJFSlUyQzlYSlBJSCZlbmNyeXB0ZWRBZElkPUEwNjMwMTYyMzhJNzNMOTA5WldIOSZ3aWRnZXROYW1lPXNwX2F0Zl9icm93c2UmYWN0aW9uPWNsaWNrUmVkaXJlY3QmZG9Ob3RMb2dDbGljaz10cnVl
        
#  https://www.amazon.in/Hammer-Wireless-Bluetooth-Playtime-Headphone/product-reviews/B07WY5M3KT/ref=lp_14583473031_1_3_sspa?s=electronics&ie=UTF8&qid=1597608129&sr=1-3-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEyOURIRjFRSUlTNUgyJmVuY3J5cHRlZElkPUEwNzM0OTMyMzJFSlUyQzlYSlBJSCZlbmNyeXB0ZWRBZElkPUEwNjMwMTYyMzhJNzNMOTA5WldIOSZ3aWRnZXROYW1lPXNwX2F0Zl9icm93c2UmYWN0aW9uPWNsaWNrUmVkaXJlY3QmZG9Ob3RMb2dDbGljaz10cnVl?ie=UTF8&reviewerType=all_reviews&pageNumber=15        
    

In [4]:
#### http://www.amazon.com/dp/{$ASIN}

#### Importing packages

In [127]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

# to ignore SSL certificate errors
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

#### Amazon reviews scrapper

In [280]:
number = 'B086RTJ1B6'
# url = "https://www.amazon.in/dp/{}".format(number)
url = "https://www.amazon.in/dp/product-reviews/{}".format(number)
url = url+"?pageNumber=4"

In [281]:
agg_titles = []
agg_names = []
agg_ratings = []
agg_verified = []
agg_dates = []
agg_contents = []

In [282]:
def helper(content, tag, parameter_key, parameter_value):
    attribute_lst = []
    attributes = content.find_all(tag, {parameter_key: parameter_value})
    for attribute in attributes:
        attribute_lst.append(attribute.contents[0])
    return attribute_lst

In [283]:
def scrapper(url):
    
    # amazon blocks requests that does not come from browser, therefore need to mention user-agent
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
    response = requests.get(url, verify=False, headers=headers)
    
    if (response.status_code != 200):
        print ("Not able to scrape ({})".format(response))
    

    # parsing content
    soup = BeautifulSoup(response.text, 'html.parser')
    ## reviews section
    reviews = soup.findAll("div", {"class":"a-section review aok-relative"})
    ## parsing reviews section
    reviews = BeautifulSoup('<br/>'.join([str(tag) for tag in reviews]), 'html.parser')
    
    
    ## 1. title
    titles = reviews.find_all("a", class_="review-title")
    title_lst = []
    for title in titles:
        title_lst.append(title.find_all("span")[0].contents[0])
        
    ## 2. name
    name_lst = helper(reviews, "span", "class", "a-profile-name")
    
    ## 3. rating
    rating_lst = helper(reviews, "span", "class", "a-icon-alt")
    
    ## 4. verified
    verified_lst = helper(reviews, "span", "data-hook", "avp-badge")
    
    ## 5. date
    date_lst = helper(reviews, "span", "data-hook", "review-date")   
    
    ## 6. content
    contents = reviews.find_all("span", {"data-hook":"review-body"})
    content_lst = []
    for content in contents:
        text_ = content.find_all("span")[0].get_text("\n").strip()
        text_ = ". ".join(text_.splitlines())
        text_ = re.sub(' +', ' ', text_)
        content_lst.append(text_)
    
    # adding to the main list
    agg_titles.extend(title_lst)
    agg_names.extend(name_lst)
    agg_ratings.extend(rating_lst)
    agg_verified.extend(verified_lst)
    agg_dates.extend(date_lst)
    agg_contents.extend(content_lst)
    
    # checking whether scrapping is completed
    if (len(title_lst) == 0):
        return "completed"
    else:
        return "running"

In [288]:
amazon_site = input("Amazon site (example: amazon.in): ")
amazon_site = "amazon.in"
# product_id = input("Product Id: ")
product_id = "B086RTJ1B6"

url = "https://www." + amazon_site + "/dp/product-reviews/" + product_id + "?pageNumber={}"

page = 1
while(True):
    
    print ("Scrapping page: {}".format(page), end="\r")
    status = scrapper(url.format(page))
    
    if (status == "completed"):
        break
    else:
        page += 1

Amazon site (example: amazon.in): amazon.in


In [285]:
reviews_df = pd.DataFrame({"date": agg_dates,
                           "name": agg_names,
                           "title": agg_titles,
                           "content": agg_contents,
                           "rating": agg_ratings})

In [286]:
reviews_df.shape

(0, 5)

In [287]:
reviews_df

,date,name,title,content,rating


In [ ]:
#no